https://docs.pytorch.org/tutorials/
https://docs.pytorch.org/tutorials/beginner/basics/intro.htmlm

In [ ]:
# PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.
# always try use conda to install packages first, if not available, then use pip
%pip install --upgrade Pillow

!conda uninstall pillow

!conda install pillow #  settle environment issue , edition issue


In [ ]:
#重装后，在 Python 中运行： sometine restart   the kernel will helpful  
from PIL import Image
print(Image.__version__)

10.4.0


In [ ]:
# 激活你的环境（如 pytorch_GPU_cuda）
!conda activate pytorch_GPU_cuda # ! call system shell to activate environment  
# 卸载
!conda uninstall torchvision

# 重新安装（与 PyTorch 版本匹配）
!conda install torchvision -c pytorch

In [1]:
import torchvision
print(torchvision.__version__)
print(torchvision.__file__)  # 查看安装位置

: 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


: 

In [1]:

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


: 

In [1]:
# PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

NameError: name 'datasets' is not defined